In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # Reduce fragmentation

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torch.utils.data import DataLoader

# Use CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset path
train = "/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"

# Load pretrained model
model = models.efficientnet_v2_l(pretrained=True)

# Modify classifier for your dataset
num_classes = len(os.listdir(train))
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model= nn.DataParallel(model).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001,betas=(0.5, 0.999))  # 10x smaller learning rate


# Optional: Mixed precision to reduce memory usage
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()

# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset and DataLoader
train_dataset = datasets.ImageFolder(root=train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=2,pin_memory=True)  # ⬅️ Reduced batch size

# Training loop
num_epochs = 15
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        with autocast():  # ⬅️ Mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
    torch.save(model.state_dict(),f"{avg_loss} {epoch}.pth")
# Save model
print("Model saved to efficientnet_model.pth")
